**README**  
This program is used to train Faster R-CNN using detectron2  
just run the forms sequentially excluding the clear dataset catalog (as thats used in case of error) to train your model  
fill in the appropriate values in the form fields for the model trying to be trained

In [ ]:
#@title connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#setup files for training

In [ ]:
#@title setup detectron2
# install dependencies: 
!pip install pyyaml==5.1
!pip install torch==1.7
!pip install torchvision==0.8.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)

     |████████████████████████████████| 276kB 12.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=f22434c22cc5b14588c9be916976fc5eef2ac702111b46a7a654f3d69ad04216
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 776.8MB 22kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 12.7MB 13.6MB/s 
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling t

In [ ]:
#@title setup variables and imports
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random, time, ntpath
from shutil import copyfile
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


def print_time(log):
  current_time = time.strftime("%H:%M:%S", time.localtime())
  print(current_time+": "+log)



#base path for training images
#@markdown path to LISA directory
LISA_path = "/content/drive/MyDrive/thesis/LISA dataset/" #@param {type:"string"}
#@markdown path where Detectron2 directory will be made
COCO_path = "/content/drive/MyDrive/thesis/COCO" #@param {type:"string"}
#@markdown path where Darknet directory will be made
YOLO_path = "/content/drive/MyDrive/thesis/YOLO/dataset" #@param {type:"string"}
#@markdown path where videos for getting FPS are stored
VIDEO_path = "/content/drive/MyDrive/thesis/VIDEOS" #@param {type:"string"}

#@markdown model version, increment each run
def getModel(num):
  mnum = (str)(num)
  return os.path.join(COCO_path,'model'+(str)(mnum))

model_num = 6 #@param {type:"number"}
model_path = getModel(model_num)
os.makedirs(model_path, exist_ok=True)

#make directory if dne
#os.makedirs(model_path, exist_ok=True)
trainer = None

#fill things variable for metadata
things=[]
for line in open(os.path.join(LISA_path, "categories.txt"), "r").read().split('\n'):
  things.append(line)

print_time("printing time to test")


03:31:36: printing time to test


In [ ]:
#@title write description
#@markdown please write a description of this model
desc = "Faster R-CNN Resnet 101 26000 iterations" #@param {type:"string"}
desc_file = open(os.path.join(model_path, 'desc.txt'), 'w')
desc_file.write(desc)
desc_file.close()

In [ ]:
#@title register datasets
from detectron2.structures import BoxMode 
d = ["Train", "Valid", "Test"]

data = dict()
print_time("reading data")
for h in d:
  with open(os.path.join(LISA_path, "COCO_Annotations_"+h+".json"), "r") as outfile:
      data[h.lower()] = json.load(outfile)
      outfile.close()

for h in d:
  for row in data[h.lower()]:
    row["annotations"][0]["bbox_mode"] = BoxMode.XYXY_ABS

train=data["train"]
valid=data["valid"]
test=data["test"]
print_time("finished reading data")

#register custom dataset for use
from detectron2.data import DatasetCatalog, MetadataCatalog

def get_sign_dicts_premade(data):
  return data

#data - data list in standard format
#s - string representing which dataset to use
#isCOCO - if its to be registered as a coco dataset
def registerDataset(data, s):
    DatasetCatalog.register("signs_"+s, lambda : get_sign_dicts_premade(data))
    MetadataCatalog.get("signs_"+s).set(thing_classes=things)
    print_time("registered "+s+" as a general dataset")

print_time("registering datasets")
registerDataset(train, "train")
registerDataset(valid, "valid")
registerDataset(test, "test")

sign_metadata = MetadataCatalog.get("signs_train")
print_time("datasets finished registering")

03:31:41: reading data
03:31:42: finished reading data
03:31:42: registering datasets
03:31:42: registered train as a general dataset
03:31:42: registered valid as a general dataset
03:31:42: registered test as a general dataset
03:31:42: datasets finished registering


In [ ]:
#@title clear dataset catalog if error
DatasetCatalog.clear()
MetadataCatalog.clear()
print_time("datasets cleared")

21:02:57: datasets cleared


In [ ]:
#@title create custom trainer class
from detectron2.engine import DefaultTrainer

#custom trainer based on example given from facebook https://github.com/facebookresearch/detectron2/blob/master/tools/train_net.py
class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)




In [ ]:
#@title create cfg file and evaluate
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

def defineCFG(cfg, max_iter, mnum):
  cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"))
  cfg.DATASETS.TRAIN = ("signs_train",) 
  cfg.DATASETS.TEST = ("signs_valid",)
  cfg.TEST.EVAL_PERIOD=500
  cfg.DATALOADER.NUM_WORKERS = 2
  cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml")
  cfg.SOLVER.IMS_PER_BATCH = 2
  cfg.SOLVER.BASE_LR = 0.001  # pick a good LR was 0.00025
  cfg.SOLVER.MAX_ITER = max_iter    # larger iter for large dataset
  cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
  cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(sign_metadata.get("thing_classes")) #gets the number of classes specified for the dataset
  cfg.OUTPUT_DIR = model_path

cfg = get_cfg()


max_iter = 52000 #@param {type:"number"}
defineCFG(cfg, max_iter, model_num)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)

[04/17 03:35:53 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

In [ ]:
trainer.resume_or_load(resume=True)
trainer.train()

[04/17 03:35:58 d2.engine.train_loop]: Starting training from iteration 26000


/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


Streaming output truncated to the last 5000 lines.
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.677
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.725
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.725
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.662
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.784
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.879
[04/17 06:19:48 d2.evaluation.coco_evaluation]: Evaluation results for bbox: 
|   AP   |  AP50  |  AP75  |  APs   |  APm   |  APl   |
|:------:|:------:|:------:|:------:|:------:|:------:|
| 52.904 | 72.600 | 64.522 | 47.844 | 60.424 | 64.324 |
[04/17 06:19:48 d2.evaluation.coco_evaluation]: Per-category bbox AP: 
| category           | AP     | category     | AP     | category     | AP     |
|:-------------------|:-------|:-------------|:-------|:-------------|:-

In [ ]:
cfg_file = open(os.path.join(model_path, "faster-rcnn.yaml"),'w')
cfg_file.write(cfg.dump())
cfg_file.close()